In [5]:
import pandas as pd
from sqlalchemy import create_engine
import os
from time import time

In [14]:
def main():
    user = "root"
    password = "root"
    host = "localhost"
    port = 5432
    db = "ny_taxi"
    table_name = "taxi_zones"
    url = "https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv"
    
    # the backup files are gzipped, and it's important to keep the correct extension
    # for pandas to be able to open the file
    if url.endswith('.csv.gz'):
        csv_name = 'output.csv.gz'
    else:
        csv_name = 'output.csv'

    os.system(f"wget {url} -O {csv_name}")

    engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')

    df_iter = pd.read_csv(csv_name, iterator=True, chunksize=100000)

    df = next(df_iter)

    # df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    # df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

    df.head(n=0).to_sql(name=table_name, con=engine, if_exists='replace')

    df.to_sql(name=table_name, con=engine, if_exists='append')


    while True: 

        try:
            t_start = time()
            
            df = next(df_iter)

            # df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
            # df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

            df.to_sql(name=table_name, con=engine, if_exists='append')

            t_end = time()

            print('inserted another chunk, took %.3f second' % (t_end - t_start))

        except StopIteration:
            print("Finished ingesting data into the postgres database")
            break

In [15]:
main()

--2023-01-18 20:47:44--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.169.168, 52.216.30.86, 54.231.135.120, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.169.168|:443... connected.


Finished ingesting data into the postgres database


HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘output.csv’

     0K .......... ..                                         100%  110M=0s

2023-01-18 20:47:45 (110 MB/s) - ‘output.csv’ saved [12322/12322]

